In diesem Notebook werden zwei Streaming Klassifizierungen mit 10.000 Instanzen durchgeführt:

1. Klassifizierung der sea_g_t Daten mit verspäteten Eintreffen der Label
2. Klassifizierung der sea_g Daten

In [7]:
import numpy as np
import pandas as pd

from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.bayes import NaiveBayes
from skmultiflow.meta import AccuracyWeightedEnsembleClassifier
from skmultiflow.lazy import KNNADWINClassifier

from skmultiflow.data import TemporalDataStream
from skmultiflow.data.file_stream import FileStream
from skmultiflow.data.data_stream import DataStream

from skmultiflow.evaluation import EvaluatePrequentialDelayed
from skmultiflow.evaluation import EvaluatePrequential

In [5]:
import warnings
warnings.filterwarnings('ignore')

#set columns
attribute = ['att_num_0', 'att_num_1', 'att_num_2']
target = 'target_0'
timestamp = 'datetime'

#read csv
t_sea_g2 = pd.read_csv("data/t_sea_g2.csv")

#convert time to datetime
t_sea_g2[timestamp] = pd.to_datetime(t_sea_g2[timestamp])

#get X, y and time
X = t_sea_g2[attribute].values
y = t_sea_g2[target].values
time = t_sea_g2[timestamp].values

#set a delay of 1 day
delay_time = np.timedelta64(1, "D")

#create stream
t_sea_g2_stream = TemporalDataStream(X, y, time, sample_delay=delay_time, ordered=False)

#set classifier
ht = HoeffdingTreeClassifier(leaf_prediction='nb')
hat = HoeffdingAdaptiveTreeClassifier()
efdt = ExtremelyFastDecisionTreeClassifier()
arf = AdaptiveRandomForestClassifier()
snb = NaiveBayes()
awe = AccuracyWeightedEnsembleClassifier(n_estimators=15, base_estimator=NaiveBayes())
knn_adwin = KNNADWINClassifier(n_neighbors=10)

#set evaluator
evaluator = EvaluatePrequentialDelayed(
                                max_samples=X.shape[0],
                                n_wait=10,
                                pretrain_size=100,
                                output_file='results_t_sea_g2.csv',
                                metrics=['accuracy', 'kappa', 
                                         'precision', 'recall', 
                                         'f1', 'running_time', 
                                         'model_size'])

#evaluate
evaluator.evaluate(stream=t_sea_g2_stream, 
                   model=[ht, hat, efdt, arf, 
                          snb, awe, knn_adwin], 
                   model_names=['Hoeffding Tree', 'Hoeffding Adaptive Tree', 
                                'Extremely Fast Decision Tree', 
                                'Adaptive Random Forest', 
                                'Naive Bayes', 'Accuracy Weighted Ensembler', 'KNN ADWIN' ])

Prequential Evaluation Delayed
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 ###################- [95%] [440.69s]Processed samples: 10000
Mean performance:
Hoeffding Tree - Accuracy     : 0.6966
Hoeffding Tree - Kappa        : 0.3684
Hoeffding Tree - Precision: 0.7254
Hoeffding Tree - Recall: 0.7709
Hoeffding Tree - F1 score: 0.7475
Hoeffding Tree - Training time (s)  : 0.63
Hoeffding Tree - Testing time  (s)  : 1.10
Hoeffding Tree - Total time    (s)  : 1.73
Hoeffding Tree - Size (kB)          : 26.6875
Hoeffding Adaptive Tree - Accuracy     : 0.6905
Hoeffding Adaptive Tree - Kappa        : 0.3692
Hoeffding Adaptive Tree - Precision: 0.7451
Hoeffding Adaptive Tree - Recall: 0.7124
Hoeffding Adaptive Tree - F1 score: 0.7284
Hoeffding Adaptive Tree - Training time (s)  : 2.55
Hoeffding Adaptive Tree - Testing time  (s)  : 0.85
Hoeffding Adaptive Tree - Total time    (s)  : 3.40
Hoeffding Adaptive Tree - Size (kB)          : 101.4863
Extremely Fast Decision Tree -

[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nb', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False, nominal_attributes=None,
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_preprune=False, nominal_attributes=None,
                                 random_state=None, remove_poor_atts=False,
                                 split_confidence=1e-07,
                        

In [9]:
#drop timestamp
t_sea_g2 = pd.read_csv('data/t_sea_g2.csv')
sea_g2 = t_sea_g2.drop(['datetime'], axis=1)

#create stream
sea_g2_stream = DataStream(sea_g2)

#set classifier
ht = HoeffdingTreeClassifier(leaf_prediction='nb')
hat = HoeffdingAdaptiveTreeClassifier()
efdt = ExtremelyFastDecisionTreeClassifier()
arf = AdaptiveRandomForestClassifier()
snb = NaiveBayes()
awe = AccuracyWeightedEnsembleClassifier(n_estimators=15, base_estimator=NaiveBayes())
knn_adwin = KNNADWINClassifier(n_neighbors=10)

#set evaluator
evaluator = EvaluatePrequential(max_samples=10000,
                                n_wait=10,
                                pretrain_size=100,
                                metrics=['accuracy', 'kappa', 
                                         'precision', 'recall', 
                                         'f1', 'running_time', 
                                         'model_size'])

#evaluate
evaluator.evaluate(stream=sea_g2_stream, 
                   model=[ht, hat, efdt, arf, 
                          snb, awe, knn_adwin], 
                   model_names=['Hoeffding Tree', 'Hoeffding Adaptive Tree', 
                                'Extremely Fast Decision Tree', 
                                'Adaptive Random Forest', 
                                'Naive Bayes', 'Accuracy Weighted Ensembler', 'KNN ADWIN'])

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 #################### [100%] [210.57s]
Processed samples: 10000
Mean performance:
Hoeffding Tree - Accuracy     : 0.7298
Hoeffding Tree - Kappa        : 0.4358
Hoeffding Tree - Precision: 0.7485
Hoeffding Tree - Recall: 0.8073
Hoeffding Tree - F1 score: 0.7768
Hoeffding Tree - Training time (s)  : 0.57
Hoeffding Tree - Testing time  (s)  : 0.91
Hoeffding Tree - Total time    (s)  : 1.48
Hoeffding Tree - Size (kB)          : 32.6250
Hoeffding Adaptive Tree - Accuracy     : 0.7344
Hoeffding Adaptive Tree - Kappa        : 0.4442
Hoeffding Adaptive Tree - Precision: 0.7494
Hoeffding Adaptive Tree - Recall: 0.8172
Hoeffding Adaptive Tree - F1 score: 0.7818
Hoeffding Adaptive Tree - Training time (s)  : 2.71
Hoeffding Adaptive Tree - Testing time  (s)  : 0.79
Hoeffding Adaptive Tree - Total time    (s)  : 3.51
Hoeffding Adaptive Tree - Size (kB)          : 100.9941
Extremely Fast Decision Tree - Accur

[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nb', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False, nominal_attributes=None,
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_preprune=False, nominal_attributes=None,
                                 random_state=None, remove_poor_atts=False,
                                 split_confidence=1e-07,
                        